In [53]:
import pandas as pd
from fastai.vision.all import *

learn = load_learner("model/model.pkl")

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


In [54]:
# Load the CSV
df = pd.read_csv("data/bodyfat_dataset.csv")

# OPTION A: Remove by index (e.g., delete row 10)
df = df.drop(index=176)

# OPTION B: Remove by condition (e.g., remove row where filename == '1_image_1.jpg')
# df = df[df['filename'] != '1_image_1.jpg']

# Save back to the same file
df.to_csv("data/bodyfat_datafat.csv", index=False)

In [55]:
def create_regression_csv(df, output_csv="data/image_labels.csv", label_col="meanPrediction", image_prefix="image_", output_dir="images"):
    # Ensure column names are stripped of whitespace
    df.columns = df.columns.str.strip()
    
    image_cols = [col for col in df.columns if col.startswith(image_prefix)]
    records = []

    for idx, row in df.iterrows():
        label = row[label_col]
        for col in image_cols:
            url = row.get(col)
            if isinstance(url, str) and url.startswith("http"):
                ext = url.split('.')[-1].split('?')[0].lower()
                ext = ext if ext in ['jpg', 'jpeg', 'png', 'webp'] else 'jpg'
                filename = f"{idx}_{col}.{ext}"
                records.append({"filename": filename, "target": label})
    
    df_out = pd.DataFrame(records)
    df_out.to_csv(output_csv, index=False)
    print(f"Created {output_csv} with {len(df_out)} labeled images")
    return df_out

In [56]:
create_regression_csv(df)

Created data/image_labels.csv with 1596 labeled images


,filename,target
0,0_image_1.jpg,8.0
1,0_image_2.jpg,8.0
2,0_image_3.jpg,8.0
3,0_image_4.jpg,8.0
4,1_image_1.jpg,9.8
...,...,...
1591,801_image_1.jpg,12.0
1592,802_image_1.jpg,10.5
1593,803_image_1.jpg,15.0
1594,804_image_1.jpg,12.0


In [57]:
from fastai.vision.all import PILImage, show_image
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt


In [58]:
df = pd.read_csv('data/image_labels_cropped.csv')  # adjust path as needed
df

,filename,target,cropped_path
0,0_image_1.jpg,8.0,NaN
1,0_image_2.jpg,8.0,NaN
2,0_image_3.jpg,8.0,NaN
3,0_image_4.jpg,8.0,cropped_images/0_image_4.jpg
4,1_image_1.jpg,9.8,cropped_images/1_image_1.jpg
...,...,...,...
1591,801_image_1.jpg,12.0,NaN
1592,802_image_1.jpg,10.5,NaN
1593,803_image_1.jpg,15.0,NaN
1594,804_image_1.jpg,12.0,cropped_images/804_image_1.jpg


In [ ]:
errors = []

# Create tqdm progress bar
pbar = tqdm(df.iterrows(), total=len(df), desc="Processing images", leave=True)

for i, row in pbar:
    try:
        img_path = f"{row['cropped_path']}"  # change if your image path differs
        img = PILImage.create(img_path)
        bf, _, probs = learn.predict(img)
        pred = float(probs[0])
        actual = float(row['target'])
        error = abs(pred - actual)

        errors.append({
            "img_path": img_path,
            "img": img,
            "actual": actual,
            "pred": pred,
            "error": error
        })

        # Optionally update progress bar postfix
        pbar.set_postfix({"last_error": f"{error:.2f}"})

    except Exception as e:
        pbar.write(f"Skipping {row['cropped_path']} due to error: {e}")

# Sort and show top 10 worst predictions
top_errors = sorted(errors, key=lambda x: x['error'], reverse=True)[:10]

for idx, item in enumerate(top_errors):
    print(f"\n#{idx+1}")
    print(f"File: {item['img_path']}")
    print(f"Predicted: {item['pred']:.2f}")
    print(f"Actual: {item['actual']:.2f}")
    print(f"Error: {item['error']:.2f}")
    show_image(item['img'], title=f"#{idx+1}: Error {item['error']:.2f}")


Processing images:   0%|                                                                                       | 0/1596 [00:00<?, ?it/s]

Skipping nan due to error: [Errno 2] No such file or directory: 'nan'
Skipping nan due to error: [Errno 2] No such file or directory: 'nan'
Skipping nan due to error: [Errno 2] No such file or directory: 'nan'


Processing images:   0%|▏                                                             | 4/1596 [00:00<01:03, 25.14it/s, last_error=2.04]

Processing images:   0%|▏                                                             | 4/1596 [00:00<01:03, 25.14it/s, last_error=4.61]

Skipping nan due to error: [Errno 2] No such file or directory: 'nan'


...

Skipping nan due to error: [Errno 2] No such file or directory: 'nan'


Processing images:  93%|███████████████████████████████████████████████████████    | 1489/1596 [01:22<00:09, 11.07it/s, last_error=1.05]

Processing images:  93%|███████████████████████████████████████████████████████    | 1489/1596 [01:22<00:09, 11.07it/s, last_error=7.29]

Processing images:  93%|███████████████████████████████████████████████████████    | 1491/1596 [01:23<00:10, 10.18it/s, last_error=4.41]

Skipping nan due to error: [Errno 2] No such file or directory: 'nan'
